# 4. 피드 포워드 신경망 
## 4.1. 다층 퍼셉트론 (MLP: Multi-Layered Perceptron)
### 4.1.2. 파이토치로 MLP 구현하기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

seed = 1337

torch.manual_seed(seed)

In [2]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

MLP 객체 생성

In [3]:
batch_size = 2 # 한 번에 입력할 샘플 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


랜덤한 입력으로 MLP 테스트

In [5]:
def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))
    print()
    
# 입력
x_input = torch.rand(batch_size, input_dim)
describe(x_input)

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.4311, 0.1362, 0.7198],
        [0.4646, 0.5330, 0.9044]])

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[-0.3970,  0.0865,  0.2468, -0.2528],
        [-0.4767,  0.0802,  0.3464, -0.3154]], grad_fn=<AddmmBackward0>)



MLP 분류기로 확률 출력하기
- 출력 텐서의 각 열은 해당 데이터 포인트에 대한 최종 특성 벡터
- --> 예측 벡터 --> 확률 분포에 대응한다 (in 분류 문제)
- 큰 양수값이 높은 확률, 낮은 음수값이 낮은 확률

In [7]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[0.1760, 0.2855, 0.3351, 0.2033],
        [0.1613, 0.2816, 0.3675, 0.1896]], grad_fn=<SoftmaxBackward0>)

